## Day 9: Smoke Basin

These caves seem to be lava tubes. Parts are even still volcanically active; small hydrothermal vents release smoke into the caves that slowly settles like rain.

If you can model how the smoke flows through the caves, you might be able to avoid it and be that much safer. The submarine generates a heightmap of the floor of the nearby caves for you (your puzzle input).

Smoke flows to the lowest point of the area it's in. For example, consider the following heightmap:

```
2199943210
3987894921
9856789892
8767896789
9899965678
```

Each number corresponds to the height of a particular location, where 9 is the highest and 0 is the lowest a location can be.

Your first goal is to find the low points - the locations that are lower than any of its adjacent locations. Most locations have four adjacent locations (up, down, left, and right); locations on the edge or corner of the map have three or two adjacent locations, respectively. (Diagonal locations do not count as adjacent.)

In the above example, there are four low points, all highlighted: two are in the first row (a 1 and a 0), one is in the third row (a 5), and one is in the bottom row (also a 5). All other locations on the heightmap have some lower adjacent location, and so are not low points.

The risk level of a low point is 1 plus its height. In the above example, the risk levels of the low points are 2, 1, 6, and 6. The sum of the risk levels of all low points in the heightmap is therefore 15.

Find all of the low points on your heightmap. What is the sum of the risk levels of all low points on your heightmap?



In [12]:
import numpy as np

def read_heightmap(filename):
    """
    Read heightmap file into numpy matrix
    
    
    """
    
    with open(filename, "r") as infile:
        height_raw = [list(map(int, l.strip())) for l in infile]
        
    # Pad with 9s around the edges to correctly treat points on the edges - 
    # points outside the map always considered higher
    height_raw = [[9]*len(height_raw[0])] + height_raw
    height_raw += [height_raw[0]]
    height_raw = [[9] + x + [9] for x in height_raw]
    
    # Convert to numpy array for ease of calculations
    height_matrix = np.matrix(height_raw)
        
    return height_matrix

def find_lowest_points(filename):
    """
    Find lowest points in heightmap

    
    """

    height_matrix = read_heightmap(filename)
    
    print(height_matrix)
    
    return 0
    
find_lowest_points("day9-example.txt")


[[9 9 9 9 9 9 9 9 9 9 9 9]
 [9 2 1 9 9 9 4 3 2 1 0 9]
 [9 3 9 8 7 8 9 4 9 2 1 9]
 [9 9 8 5 6 7 8 9 8 9 2 9]
 [9 8 7 6 7 8 9 6 7 8 9 9]
 [9 9 8 9 9 9 6 5 6 7 8 9]
 [9 9 9 9 9 9 9 9 9 9 9 9]]


0